# Step 2 - Data Wrangling Raw Data in Local Data Lake to Digestable Data  
Loading, merging, cleansing, unifying and wrangling Gartner IT Symposium Session Data from still fairly raw JSON files in the local datalake. 

The gathering of raw data from the (semi-)public API for the Session Catalog into a local data lake was discussed and performed  in <a href="./1-GartnerITSymposiumXpo2020-Session Catalog - retrieving raw session data in JSON files.ipynb">1-GartnerITSymposiumXpo2020-Session Catalog - retrieving raw session data in JSON files</a>. The current notebook starts from the raw JSON file in local directory `./datalake`.

This notebook describes how to load, combine and wrangle the data from this file. This notebook shows for example how to load data from a JSON file, discard undesired attributes, deduplicate the record set, derive attributes for easier business intelligence & machine learning and write the resulting data set to a single JSON file.

Steps in this notebook:
* Load from raw JSON
    * <a href="#deduplicate">discard redundant columns</a>
* <a href="#explore">Explore Data Frame</a>
* <a href="#enriching">Enrich Data</a>
* <a href="publish">Publish Wrangle Results</a>

The deliverable from this notebook is a single file `gartner-it-synmposium-xpo-2020-sessions-wrangled.json` in the `datawarehouse` folder. This file contains unique, filtered, enriched data that is in a poper shape to perform further analysis on.

# Load data from raw JSON file
This first section describes how the session data from Gartner IT Symposium Xpo 2020 is loaded from a file with the raw JSON session data. This file was produced by a different notebook (<a href="./1-GartnerITSymposiumXpo2020-Session Catalog - retrieving raw session data in JSON files.ipynb">1-GartnerITSymposiumXpo2020-Session Catalog - retrieving raw session data in JSON files</a>) from the Gartner IT Symposium Session Catalog API.

The file is read into a Pandas Data Frame object - called *ss* (session schedule).

Let's start with reading the session information from a single file into a Pandas Data Frame - to get a feel for how that works and what it results in.

In [53]:
#read a single session data file and parse the JSON content into a Pandas Data Frame
import pandas as pd
import json

dataLake = "datalake/" # file system directory used for storing the gathered data

ss = pd.read_json("{0}gartner-it-symposium-xpo-2020.json".format(dataLake))
ss.head(3)


,sessionID,externalID,code,abbreviation,title,abstract,type,status,length,modified,...,times,attributevalues,featured_value,useWaitingList,es_metadata_id,highlight,videos,date,time,sponsors
0,1597610066073001MIKP,2237883,K1,K1,Gartner Opening Keynote: Seize the Moment to C...,Leaders worldwide are guiding their organizati...,Keynote,Accepted,45,2020-11-09 10:49:09+00:00,...,"[{'sessionTimeID': '1597610066073002MG6k', 'ex...","[{'value': 'All Sessions', 'attributevalue_id'...",1,1,1597610066073001MIKP,{},[],2020-11-09,10:00,None
1,1599064516896001RRlL,2254587,11E,11E,Composable Businesses Need Antifragile Strategies,"In order to win in volatile times, composabili...",Track Sessions,Accepted,30,2020-11-09 16:12:38+00:00,...,"[{'sessionTimeID': '1599064516896002RuZ1', 'ex...","[{'value': 'All Sessions', 'attributevalue_id'...",1,1,1599064516896001RRlL,{},[],2020-11-09,11:00,None
2,1599061799403001Zhvo,2247633,11a,11a,Postpandemic Planning of IT Strategy,As CIOs come out of the immediate response pha...,Track Sessions,Accepted,30,2020-11-09 16:08:52+00:00,...,"[{'sessionTimeID': '1599061799403002ZebZ', 'ex...","[{'value': 'All Sessions', 'attributevalue_id'...",1,1,1599061799403001Zhvo,{},[],2020-11-09,11:00,None


All session data is loaded into a single data frame. We will use `ss` as the sink - the data frame into which all sessions records are to be loaded.

### Some key  metrics on the merged Sessions Set
The shape function on the data frame returns the dimensions of the frame: the number of rows by the number of columns:

In [54]:
ss.shape

(136, 44)

In [55]:
# total memory usage
ss.memory_usage(index=True, deep=True).sum()

396260

In [56]:
#list all columns in the Dataframe
ss.columns

Index(['sessionID', 'externalID', 'code', 'abbreviation', 'title', 'abstract',
       'type', 'status', 'length', 'modified', 'published',
       'hasWebinarProfile', 'webinarProfile', 'webinarProvider',
       'embedableWebinar', 'useDoubleBooking', 'allowDoubleBooking',
       'eventCode', 'eventId', 'eventName', 'viewAccessPublic', 'viewAccess',
       'scheduleAccess', 'waitlistAccess', 'waitlistLimit', 'viewFileAccess',
       'files', 'publicViewPrivateSchedule', 'title_sort', 'code_id',
       'codeParts', 'type_displayorder', 'type_displayorder_string',
       'participants', 'times', 'attributevalues', 'featured_value',
       'useWaitingList', 'es_metadata_id', 'highlight', 'videos', 'date',
       'time', 'sponsors'],
      dtype='object')

In [57]:
# data types for the columns in the data frame
ss.dtypes

sessionID                                 object
externalID                                object
code                                      object
abbreviation                              object
title                                     object
abstract                                  object
type                                      object
status                                    object
length                                     int64
modified                     datetime64[ns, UTC]
published                                  int64
hasWebinarProfile                           bool
webinarProfile                            object
webinarProvider                           object
embedableWebinar                         float64
useDoubleBooking                            bool
allowDoubleBooking                         int64
eventCode                                 object
eventId                                   object
eventName                                 object
viewAccessPublic    

In [58]:
ss.groupby(['type'])['type'].count()

type
Breaks                        4
CIO Story Sessions            6
Keynote                       4
Signature Series              4
Solution Provider Session    26
Track Sessions               92
Name: type, dtype: int64

<a name="discard" />

## Discard unneeded attributes
Early inspection of the JSON document and the session catalog website has provided insight in the available attributes and their relevance. Below, you will see an overview of all columns in the Data Frame - corresponding with the top level items in the JSON document. The subsequent step removes from the Data Frame all columns that seem irrelevant for our task at hand. 

These columns seem relevant for the web frontend developers, for planned but not realized objectives or for unknown purposes. In order to not carry more weight than necessary - because of performance, resource usage and lurking complexity we get rid of columns that seem irrelevant. If we need them after all, they are still available in the data lake.    

Remove the unwanted columns from the Dataframe
(allowDoubleBooking .'codeParts', 'code_id', 'es_metadata_id',type_displayorder	type_displayorder_string	useDoubleBooking	useWaitingList	videos	viewAccess	viewAccessPublic, ...)


In [59]:
# remove columns
# Note that the original 'data' object is changed when inplace=True
ss.drop([
    'allowDoubleBooking'
         ,'codeParts', 'code_id', 'es_metadata_id','type_displayorder'
       ,'type_displayorder_string', 'useDoubleBooking'
       ,'useWaitingList', 'videos'
      , 'viewAccess', 'viewAccessPublic','viewFileAccess', 'waitlistAccess', 'waitlistLimit'
    ,  'eventId','eventName','featured_value','publicViewPrivateSchedule','published', 'scheduleAccess','sessionID','status'
    ,'externalID','highlight','abbreviation'
        ]
        , axis=1, inplace=True) 

<a name="deduplicate" />

# Exploring the Data
Let's briefly look at the data we now have in the Pandas Data Frame. What does the data look like? What values do we have in the columns? And what complexity is hiding in some columns with nested values, such as participants, attributevalues and files.

Note: https://morphocode.com/pandas-cheat-sheet/ provides a quick overview of commands used for inspecting and manipulating the data frame.

In [60]:
#an overview of the current contents of the data frame
ss.head(6)


,code,title,abstract,type,length,modified,hasWebinarProfile,webinarProfile,webinarProvider,embedableWebinar,eventCode,files,title_sort,participants,times,attributevalues,date,time,sponsors
0,K1,Gartner Opening Keynote: Seize the Moment to C...,Leaders worldwide are guiding their organizati...,Keynote,45,2020-11-09 10:49:09+00:00,True,intrado-tgi,intrado-show,0.0,esc32v,"[{'fileType': 'presentation files', 'fileId': ...",gartneropeningkeynoteseizethem,[{'speakerId': '15132790924060010zbe_159475592...,"[{'sessionTimeID': '1597610066073002MG6k', 'ex...","[{'value': 'All Sessions', 'attributevalue_id'...",2020-11-09,10:00,None
1,11E,Composable Businesses Need Antifragile Strategies,"In order to win in volatile times, composabili...",Track Sessions,30,2020-11-09 16:12:38+00:00,True,intrado-tgi,intrado-show,0.0,esc32v,"[{'fileType': 'presentation files', 'fileId': ...",composablebusinessesneedantifr,[{'speakerId': '1526636305622001pt8A_159475592...,"[{'sessionTimeID': '1599064516896002RuZ1', 'ex...","[{'value': 'All Sessions', 'attributevalue_id'...",2020-11-09,11:00,None
2,11a,Postpandemic Planning of IT Strategy,As CIOs come out of the immediate response pha...,Track Sessions,30,2020-11-09 16:08:52+00:00,True,intrado-tgi,intrado-show,0.0,esc32v,"[{'fileType': 'presentation files', 'fileId': ...",postpandemicplanningofitstrate,[{'speakerId': '1516210828763001Sg6P_159475592...,"[{'sessionTimeID': '1599061799403002ZebZ', 'ex...","[{'value': 'All Sessions', 'attributevalue_id'...",2020-11-09,11:00,None
3,11C,Ten Rules for Rapid IT Spend Reduction,Difficult times call for difficult actions. In...,Track Sessions,30,2020-11-09 16:10:51+00:00,True,intrado-tgi,intrado-show,0.0,esc32v,"[{'fileType': 'presentation files', 'fileId': ...",tenrulesforrapiditspendreducti,[{'speakerId': '1538713343763001NS1Y_159475592...,"[{'sessionTimeID': '1597342478776002JnhU', 'ex...","[{'value': 'All Sessions', 'attributevalue_id'...",2020-11-09,11:00,None
4,11G,The Cloud Computing Scenario: The Future Is Di...,"Distributed cloud brings together edge, hybrid...",Track Sessions,30,2020-11-09 16:13:35+00:00,True,intrado-tgi,intrado-show,0.0,esc32v,"[{'fileType': 'presentation files', 'fileId': ...",thecloudcomputingscenariothefu,[{'speakerId': '1598377738638001HfhD_159475592...,"[{'sessionTimeID': '1599062702602002pTY2', 'ex...","[{'value': 'All Sessions', 'attributevalue_id'...",2020-11-09,11:00,None
5,11F,The Everything Customer,The 2020 crisis has only accelerated the need ...,Track Sessions,30,2020-11-09 16:13:02+00:00,True,intrado-tgi,intrado-show,0.0,esc32v,"[{'fileType': 'presentation files', 'fileId': ...",theeverythingcustomer,[{'speakerId': '1516215143135001S4Bu_159475592...,"[{'sessionTimeID': '1597342480113002JJjq', 'ex...","[{'value': 'All Sessions', 'attributevalue_id'...",2020-11-09,11:00,None


In [61]:
# and this overview of the rows and columns in the data frame.
print(ss.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 0 to 135
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   code               136 non-null    object             
 1   title              136 non-null    object             
 2   abstract           136 non-null    object             
 3   type               136 non-null    object             
 4   length             136 non-null    int64              
 5   modified           136 non-null    datetime64[ns, UTC]
 6   hasWebinarProfile  136 non-null    bool               
 7   webinarProfile     128 non-null    object             
 8   webinarProvider    128 non-null    object             
 9   embedableWebinar   128 non-null    float64            
 10  eventCode          136 non-null    object             
 11  files              132 non-null    object             
 12  title_sort         136 non-null    object         

In [62]:
#let's look at all different values for length (the duration of each session in minutes)
ss['length'].unique()
# yes - it is that simple!

array([45, 30, 20, 25, 40])

In [63]:
# and what about (session) type?
ss['type'].unique()

array(['Keynote', 'Track Sessions', 'Signature Series',
       'Solution Provider Session', 'CIO Story Sessions', 'Breaks'],
      dtype=object)

Some of the columns in the data frame contain complex, nested values. For example the `attributevalues` column. It contains a JSON array - a list of objects that each describe some attribute for the session. Examples of session attributes that are defined in this somewhat convoluted way are *VirtualSessionType*, *Day*, *Time*, *Topic*, *Industries*, *SuggestedAgendas*, *SessionType*, *Tracks* . A little later on, we will create new, proper features in the data frame based on values extracted from this complex attributevalues column - in order to make it possible to make good use of this information for visualization, analysis and machine learning. Note: some of the attribute_ids can occur more than once in a single session's attributevalues property, with different values. For example one session can have multiple tracksm, industries and suggested agendas defined against it.

In [64]:
# some of the columns or attributes have nested values. It seems useful to take a closer look at them. 
# show nested array attributevalues for first record in Dataframe - this is an attribute which contains an array of objects that each define a specific characteristic of the session, 
# such as date and time, topic, virtual session type, industries 

ss['attributevalues'][10]

[{'value': 'By attending this session, you agree to provide your contact details to the Exhibitor.',
  'attributevalue_id': '1590099041051001RLM8',
  'rf_attributevalue_id': '1590099041696001RBJv',
  'attributevalue_code': '1590099041051001RLM8',
  'dataForm': '1594755929340026fwbM',
  'published': 1.0,
  'viewAccessPublic': True,
  'viewAccess': ['1602195924883001KXok',
   '1602250829307001D3q5',
   '1602251022009001DiaV'],
  'scheduleAccess': ['1602195924883001KXok',
   '1602250829307001D3q5',
   '1602251022009001DiaV'],
  'attribute_id': 'ExhibitorSessionDisclaimer',
  'attribute': 'Exhibitor Session Disclaimer',
  'displayorder': 0.0},
 {'value': 'All Sessions',
  'attributevalue_id': '1598394051609001kD5K',
  'rf_attributevalue_id': '1598394052235001kZxf',
  'attributevalue_code': '1598394051609001kD5K',
  'dataForm': '1594755929340026fwbM',
  'published': 1.0,
  'viewAccessPublic': True,
  'viewAccess': ['1602195924883001KXok',
   '1602250829307001D3q5',
   '1602251022009001DiaV'

The *participants* column also contains a complex object. This column too contains a JSON array with the people associated with a session as speaker. The array contains a nested object for each speaker. This object has a lot of data in it - from name and biography and special designations (titles) for the speaker to company, job title, URL to a picture and details on all (other) sessions the speaker is involved in.

Let's take a look at an example.

In [65]:
#show nested array participants for 11th record in the Dataframe
ss['participants'][5]


[{'speakerId': '1516215143135001S4Bu_15947559284580esc32v',
  'userRef': '1516215143135001S4Bu',
  'firstName': 'Hung',
  'lastName': 'LeHong',
  'fullName': 'Hung LeHong',
  'bio': "Hung LeHong is a VP and Gartner Fellow in Gartner Research and a member of the CEO and Digital Business Leaders research group. He focuses on CEOs and other C-level executives to help them anticipate changes to business models and consumer trends caused by digital business. He helps executives understand what's coming in three years and beyond, so they can innovate today to improve their position or effectiveness in the future.",
  'companyName': 'Gartner',
  'jobTitle': 'Distinguished VP Analyst',
  'photoURL': 'https://emtemp.gcom.cloud/ngw/commonassets/images/headshots/experts/hung-lehong.jpg',
  'Speaker-Photo-Published': 'Published',
  'globalFirstname': 'Hung',
  'globalLastname': 'LeHong',
  'globalFullName': 'Hung LeHong',
  'globalBio': "Hung LeHong is a VP and Gartner Fellow in Gartner Research a

The *files* column too contains a JSON array. This array contains entries for files associated with the session. These files provide the slides or other supporting content for the session. Each session is supposed to have exactly one files associated with it. Some do not - for example because the speaker(s) forgot to upload their slides. 

Let's inspect the JSON contents of the *files* column. It contains the name of the file and the URL from where it can be downloaded.

In [66]:
#show nested array files
ss['files'][0]

[{'fileType': 'presentation files',
  'fileId': '1604693309737001Ibc2',
  'filename': 'Gartner Composable Business Index.pdf',
  'url': 'https://static.rainfocus.com/gartner/esc32v/sess/1597610066073001MIKP/presentation files/Gartner%20Composable%20Business%20Index_1604693309624001IHjw.pdf',
  'viewFileAccess': ['1602195924883001KXok',
   '1602250829307001D3q5',
   '1602251022009001DiaV']},
 {'fileType': 'presentation files',
  'fileId': '1604855370930001jmGZ',
  'filename': 'Gartner Opening Keynote Seize the Moment to Compose a Resilient Future EMEA.pdf',
  'url': 'https://static.rainfocus.com/gartner/esc32v/sess/1597610066073001MIKP/presentation files/Gartner%20Opening%20Keynote%20Seize%20the%20Moment%20to%20Compose%20a%20Resilient%20Future%20EMEA_1604855370764001jmaY.pdf',
  'viewFileAccess': ['1602195924883001KXok',
   '1602250829307001D3q5',
   '1602251022009001DiaV']}]

<a name="enriching" />

# Enriching the data
In this section we are engineering the data to produce some attributes or features that are easier to work with once we start doing data science activities such as business intelligence or machine learning. In this section we are not yet actually bringing in external data sources to add information we not already have in our set. However, we are making the data we already have more accessible and thereby more valuable. So in that sense, this too can be called *enriching*. 

The enrichments performed on the session data in the data frame :
* set a flag at session level to indicate whether the session is an Gartner session (with Gartner staff among the speakers- speaker has Gartner in companyName attribute)
* derive track(s) from attributevalues array (where attribute_id=Tracks)
* derive topic from attributevalues array (where attribute_id=Topic)
* derive industrie(s) from attributevalues array (where attribute_id=Industries)
* derive suggested agendas from attributevalues array (where attribute_id=SuggestedAgendas)
* set attribute with number of speakers on the session
* number of instances of the session    

### Gartner Speaker and File Uploaded
These functions `gartner_speaker` and `file_flag` are invoked for every record in the data frame. They are used to derive new, first class attributes to indicate whether or not at least one speaker working for Gartner is associated with a session (Y or N) and if a file has been upload (presumably with slides) for the session. The information represented by these two new attributes already exists in the data frame - but in a way that makes it quite unaccessible to the data analyst.

In [67]:
#function to derive Gartner flag from participants
def gartner_speaker(session):
      result = "N"
      # loop over speakers; if for any of them, companyName contains the word Gartner, the result =Y  
      if isinstance(session["participants"], list):
         for x in session["participants"] :
            if ("gartner" in x['companyName'].lower()):
                result='Y'
      return result

New columns `gartner_speaker` and `file_flag` are added to the data frame with values derived for each record by applying the functions with corresponding names.  

In [68]:
#set gartner_speaker flag
#apply function gartner_speaker to every row in the data frame to derive values for the new column gartner_speaker
ss['gartner_speaker'] = ss.apply(gartner_speaker, axis=1)
# show the values and the number of occurrences for the new column gartner_speaker
ss['gartner_speaker'].value_counts()


Y    98
N    38
Name: gartner_speaker, dtype: int64

In [69]:
#function to derive file flag from files
def file_flag(session):
      result = "N"
      if isinstance(session.get("files",None),list) :
          # loop over files; if any exist, then result = Y
          for x in session["files"][:]:
             if x['fileId']:
               result='Y'
               break
      return result

In [70]:
#set file_flag
#apply function file_flag to every row in the data frame to derive values for the new column file_flag
ss['file_flag'] = ss.apply(file_flag, axis=1)
ss['file_flag'].value_counts()


Y    132
N      4
Name: file_flag, dtype: int64

### Topic
Each session is described through a topic, defined in an attributevalue

In [71]:
#function to derive topic for a session
def session_topic(session):
      result = "N"
      # loop over attributevalues; if any exist with attribute_id == "topic"
      for x in session["attributevalues"][:]:
        if "attribute_id" in x: 
            if x["attribute_id"]=="Topic":
                result= x["value"]
                break # no point in continuing if we have found what we are looking for
      return result

This cell runs creates a new column Topic in the data frame. It will set the Topic values for each session

In [72]:
ss["topic"] = ss.apply(session_topic, args=(), axis=1) 
ss.groupby(['topic'])['topic'].count()

topic
AI, Data & Analytics                           15
Cloud, Infrastructure & Operations              7
Customer & Employee Experience                 10
Cybersecurity & Risk Management                 4
Innovation & Emerging Technologies             10
Leadership & Business Transformation           29
N                                              20
Operational Excellence and Operating Models     2
Privacy & Ethics in a Digital Society           4
Rebuilding After Crisis                        16
Smarter Spending                                5
Strategic Planning & Execution                  4
Talent, Culture & The Future of Work           10
Name: topic, dtype: int64

### Tracks
All sessions are assigned to one or more tracks. These tracks are categories that help attendees identify and assess sessions. Some examples of tracks are: TBP.

The track(s) are found in the nested object *attributevalues* under the attribute_id *Tracks*.

The next function is created to return a String array for a session with all the tracks associated with the session.

In [73]:
#function to derive track flag for a session
def session_track(session):
      result = ['abc']      
      # loop over attributevalues; if any exist with attribute_id == "Tracks"
      for x in session["attributevalues"][:]:
        if "attribute_id" in x: 
            if x["attribute_id"]=="Tracks":
               result.append( x["value"])
      del result[0]   
      return result

The next cell uses the function `session_track` to produce the value for each session for the track. The cell then prints out the tracks for a sample of sessions.

In [74]:
# add column track with a value derived from the session record
ss["track"] = ss.apply(session_track,  axis=1) 
print("Assigned Track")
ss[['title','topic','track']].tail(10)

Assigned Track


,title,topic,track
126,LRS Output Management: ABB Case Study – IT Sta...,N,[]
127,Nutanix: Four Step Journey to a Successful Hyb...,"Cloud, Infrastructure & Operations",[]
128,Energy Transition: The Next Surprising Disrupt...,Rebuilding After Crisis,[]
129,Into the Digital Future With the Composable Pu...,Leadership & Business Transformation,[]
130,The Future Of Data-Driven Transportation,"AI, Data & Analytics",[]
131,Break Through the Barriers to Scaling Agile a...,"Talent, Culture & The Future of Work",[Leadership]
132,Five Ways to Stay Focused on Customers in Unce...,Customer & Employee Experience,[Business Strategy]
133,Gartner Global Scenarios 2020: How to Accelera...,Leadership & Business Transformation,[Business Strategy]
134,Halfway to Star Trek: A Professional's Guide t...,Leadership & Business Transformation,[Technology & Information]
135,How to Staff Your AI Dream Team?,"AI, Data & Analytics",[Leadership]


### Industries and Suggested Agenda
All sessions are assigned to one or more Industries and Suggested Agendas. These are both categories that help attendees identify and assess sessions. 

The industrie(s) are found in the nested object *attributevalues* under the attribute_id *Industries* and the suggested agendas under *SuggestedAgendas* .

The next function is created to return a String array for a session with all the values for a specific attributesvaluesid associated with the session.

In [75]:
def session_attributevalues(session, attribute_id):
      result = ['abc']      
      # loop over attributevalues; if any exist with attribute_id == attribute_id
      for x in session["attributevalues"][:]:
        if "attribute_id" in x: 
            if x["attribute_id"]== attribute_id:
               result.append( x["value"])
      del result[0]   
      return result

In [76]:
ss["industries"] = ss.apply(session_attributevalues, args=("Industries",),  axis=1) 
ss["suggested_agendas"] = ss.apply(session_attributevalues, args=("SuggestedAgendas",),  axis=1) 
print("Assigned Industries")
ss[['title','topic','track',"industries","suggested_agendas"]].tail(10)

Assigned Industries


,title,topic,track,industries,suggested_agendas
126,LRS Output Management: ABB Case Study – IT Sta...,N,[],[],[]
127,Nutanix: Four Step Journey to a Successful Hyb...,"Cloud, Infrastructure & Operations",[],[],[]
128,Energy Transition: The Next Surprising Disrupt...,Rebuilding After Crisis,[],[Oil/Gas & Energy],[Business Executives]
129,Into the Digital Future With the Composable Pu...,Leadership & Business Transformation,[],[Government],[]
130,The Future Of Data-Driven Transportation,"AI, Data & Analytics",[],[Manufacturing],[Data and Analytics Leaders]
131,Break Through the Barriers to Scaling Agile a...,"Talent, Culture & The Future of Work",[Leadership],[],[Application Leaders]
132,Five Ways to Stay Focused on Customers in Unce...,Customer & Employee Experience,[Business Strategy],[],"[Business Executives, Midsize Enterprise]"
133,Gartner Global Scenarios 2020: How to Accelera...,Leadership & Business Transformation,[Business Strategy],[],"[Business Executives, Midsize Enterprise, Glob..."
134,Halfway to Star Trek: A Professional's Guide t...,Leadership & Business Transformation,[Technology & Information],[],[CIO Agenda]
135,How to Staff Your AI Dream Team?,"AI, Data & Analytics",[Leadership],[],"[CIO Agenda, Data and Analytics Leaders]"


### Number of Speakers per Session
The number of speakers making an appearance in a session is another fact that while not readily available is also hiding in our data frame. We will turn this piece of information into an explicit feature. The next cell counts the number of elements in the  participants array - and assigns it to the speaker_count attribute of each session record.

In [77]:
#set number of speakers for each session by taking the length of the list in the participants column
ss["speaker_count"] = ss['participants'].apply(lambda x: len(x) if isinstance(x, list) else 0)
# list the values in the new speaker_count column and the number of occurrences. 12 participants in one session?
ss['speaker_count'].value_counts()

1    114
0      8
2      7
3      4
4      2
5      1
Name: speaker_count, dtype: int64

### Number of instances per session
Most sessions are scheduled just once. However, sessions could be executed multiple times. This can be derived from the *times* column in the data frame, simply by taking the number of elements in the JSON array in that column.

The next cell adds a column to the data frame, with for each session the number of times that session takes place.

In [78]:
#set number of instancesfor each session by taking the length of the list in the times column
ss["instance_count"] = ss['times'].apply(lambda x: len(x) if x else None)
ss['instance_count'].value_counts()

1    136
Name: instance_count, dtype: int64

## Persist Pandas Dataframe - as single, consolidated, reshaped, enriched JSON file

One thing we want to be able to do with the data we gather, is to persist it for future use by data analists, data scientists and other stakeholders. We could store the data in a NoSQL database, a cloud storage service or simply as a local file. For now, let's do the latter: store the cleansed and reshaped data in a local JSON file for further enrichment, visualization and machine learning purposes.

The file is called `gartner-it-synmposium-xpo-2020-sessions-wrangled.json` and it will be stored in the `datawarehouse` folder.

In [90]:
dataWarehouse = "datawarehouse/" # file system directory used for storing the wrangled data
ss.to_json("{0}gartner-it-synmposium-xpo-2020-sessions-wrangled.json".format(dataWarehouse), force_ascii=False)

A quick check to see whether the wrangled session data was successfully written to disk - and can be read again. If we can read it, than we can safely assume that in the next phase in the data analytics flow the same will succeed.

In [91]:
dubss = pd.read_json("{0}gartner-it-synmposium-xpo-2020-sessions-wrangled.json".format(dataWarehouse))
dubss.head(10)

,code,title,abstract,type,length,modified,hasWebinarProfile,webinarProfile,webinarProvider,embedableWebinar,...,time,sponsors,gartner_speaker,file_flag,topic,track,industries,suggested_agendas,speaker_count,instance_count
0,K1,Gartner Opening Keynote: Seize the Moment to C...,Leaders worldwide are guiding their organizati...,Keynote,45,2020-11-09 10:49:09,True,intrado-tgi,intrado-show,0.0,...,10:00,None,Y,Y,Leadership & Business Transformation,"[Technology & Information, Leadership, Busines...","[Oil/Gas & Energy, Education, Utilities, Healt...","[Application Leaders, CIO Agenda, Data and Ana...",4,1
1,11E,Composable Businesses Need Antifragile Strategies,"In order to win in volatile times, composabili...",Track Sessions,30,2020-11-09 16:12:38,True,intrado-tgi,intrado-show,0.0,...,11:00,None,Y,Y,Leadership & Business Transformation,[Business Strategy],[],[Business Executives],1,1
2,11a,Postpandemic Planning of IT Strategy,As CIOs come out of the immediate response pha...,Track Sessions,30,2020-11-09 16:08:52,True,intrado-tgi,intrado-show,0.0,...,11:00,None,Y,Y,Rebuilding After Crisis,[Business Strategy],[],[Senior IT Leaders],1,1
3,11C,Ten Rules for Rapid IT Spend Reduction,Difficult times call for difficult actions. In...,Track Sessions,30,2020-11-09 16:10:51,True,intrado-tgi,intrado-show,0.0,...,11:00,None,Y,Y,Smarter Spending,[Business Strategy],[],"[CIO Agenda, Business Executives]",1,1
4,11G,The Cloud Computing Scenario: The Future Is Di...,"Distributed cloud brings together edge, hybrid...",Track Sessions,30,2020-11-09 16:13:35,True,intrado-tgi,intrado-show,0.0,...,11:00,None,Y,Y,"Cloud, Infrastructure & Operations",[Technology & Information],[],[IOCS Leaders],1,1
5,11F,The Everything Customer,The 2020 crisis has only accelerated the need ...,Track Sessions,30,2020-11-09 16:13:02,True,intrado-tgi,intrado-show,0.0,...,11:00,None,Y,Y,Customer & Employee Experience,[Business Strategy],[],"[Midsize Enterprise, Global Enterprise Executi...",1,1
6,11H,The Top Strategic Technology Trends for 2021,Strategic technology trends have significant d...,Signature Series,30,2020-11-09 16:14:06,True,intrado-tgi,intrado-show,0.0,...,11:00,None,Y,Y,Innovation & Emerging Technologies,"[Technology & Information, Leadership, Busines...","[Oil/Gas & Energy, Education, Utilities, Healt...","[Application Leaders, Security and Risk Leader...",1,1
7,11D,Who Do You Want to Be Post-Crisis? Actions for...,There is a lot of corporate soul searching goi...,Track Sessions,30,2020-11-09 16:12:12,True,intrado-tgi,intrado-show,0.0,...,11:00,None,Y,Y,Leadership & Business Transformation,[Leadership],[],[Senior IT Leaders],1,1
8,11B,You Do Not Need a New IT Organizational Design...,"""How to organize for digital business"" is ofte...",Track Sessions,30,2020-11-09 16:09:59,True,intrado-tgi,intrado-show,0.0,...,11:00,None,Y,Y,Operational Excellence and Operating Models,[Leadership],[],[IOCS Leaders],1,1
9,ESCSPS1,AWS: Digital Transformation – Mental Models fo...,Enterprise transformations are hard. Almost ev...,Solution Provider Session,20,2020-11-09 16:15:56,True,intrado-tgi,intrado-show,0.0,...,11:00,[1511981509873001Vgfv],N,Y,N,[],[],[],1,1


If and when the previous cell lists session records correctly, then the data warehouse has been populated with the consolidated file *oow2018-sessions-wrangled.json* with all sessions - cleansed, deduplicated, enriched and ready for further processing.

The wrangled data set no longer contains many of the attributes not conceivably useful for further analysis. It has been extended with (derived) attributes that will probably be useful for next data analytics tasks. Additionally, the record set has been deduplicated to only the uniqe sessions.   

In [92]:
# a quick list of all columns
dubss.columns

Index(['code', 'title', 'abstract', 'type', 'length', 'modified',
       'hasWebinarProfile', 'webinarProfile', 'webinarProvider',
       'embedableWebinar', 'eventCode', 'files', 'title_sort', 'participants',
       'times', 'attributevalues', 'date', 'time', 'sponsors',
       'gartner_speaker', 'file_flag', 'topic', 'track', 'industries',
       'suggested_agendas', 'speaker_count', 'instance_count'],
      dtype='object')